Chowdhury Walid Tahabee
#### APAN5400: MANAGING DATA
##### 03/09/2021
### Assignment 6: Graph Databases - Neo4j

# Installing required packages

In [1]:
!pip3 install neo4j
!pip3 install py2neo

# Connecting to Neo4j

In [2]:
from neo4j import GraphDatabase

database_name = "apan5400"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/"+database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


# Importing companies.csv

In [4]:
query1 = (
        "LOAD CSV WITH HEADERS FROM 'file:///companies.csv' AS line "
        "CREATE (company:Company { \
                       name: line.companyName, \
                  employees: toInteger(line.employees),\
                       year: toInteger(line.founded), \
                    revenue: toFloat(line.annualRevenue) \
                            } \
            )"
     "MERGE  (city:City {name: line.city})"
     "MERGE  (state:State {name: line.state})"
     "MERGE  (country:Country {name: line.country})"    
     "CREATE (company)-[:HEADQUARTERED_IN]->(city)"
     "CREATE (city)-[:LOCATED_IN]->(state)"
     "CREATE (state)-[:PART_OF]->(country)"
    )

result1 = session.run(query1)
print("All companies are imported from a csv file!")

All companies are imported from a csv file!


# Selecting and printing all nodes in companies.csv

In [5]:
result2 = session.run("MATCH (n) RETURN n as nodes")
[record["nodes"] for record in result2]

[<Node id=0 labels=frozenset({'Company'}) properties={'name': 'Apple', 'revenue': 274.5, 'employees': 147000, 'year': 1976}>,
 <Node id=1 labels=frozenset({'City'}) properties={'name': 'Cupertino'}>,
 <Node id=2 labels=frozenset({'State'}) properties={'name': 'California'}>,
 <Node id=3 labels=frozenset({'Country'}) properties={'name': 'USA'}>,
 <Node id=4 labels=frozenset({'Company'}) properties={'name': 'Microsoft', 'revenue': 143.0, 'employees': 166475, 'year': 1975}>,
 <Node id=5 labels=frozenset({'City'}) properties={'name': 'Redmond'}>,
 <Node id=6 labels=frozenset({'State'}) properties={'name': 'Washington'}>,
 <Node id=7 labels=frozenset({'Company'}) properties={'name': 'Amazon', 'revenue': 386.0, 'employees': 1298000, 'year': 1994}>,
 <Node id=8 labels=frozenset({'City'}) properties={'name': 'Seattle'}>,
 <Node id=9 labels=frozenset({'Company'}) properties={'name': 'Google', 'revenue': 187.5, 'employees': 135301, 'year': 1998}>,
 <Node id=10 labels=frozenset({'City'}) properti

# Counting the number of nodes

In [48]:
result3 = session.run("MATCH (n) RETURN count(n) as nodes_count")
[record["nodes_count"] for record in result3]

[27]

The graph database was imported from the 'companies.csv' file. The 'companyName', 'employees', 'founded', and 'annualRevenue' columns in the csv file were each imported as properties under the 'Company' node label using the CREATE clause and converted to appropriate data types.
The 'city', 'state' and 'country' columns were each imported under their own corresponding node labels with the values mapped to their 'name' property using the MERGE clause.

##### 1. Node label: 
Companies

##### Properties: 

i. name (imported from 'companyName' column in csv file) (string)

ii. employees (imported from 'employees' column in csv file) (int)

iii. year (imported from 'founded' column in csv file) (int)

iv. revnue (imported from 'annualRevenue column in csv file) (float)

##### 2. Node label:
City
##### Properties: 
name (imported from the city column in csv file) (string)

##### 3. Node label:
State
##### Properties: 
name (imported from the state column in csv file) (string)

##### 4. Node label:
Country
##### Properties:
name (imported from the country column in csv file) (string)


3 relationships were generated using the CREATE clause, as well as the '-' and '->' operators to specify the direction of the relationship: 
##### 1.  HEADQUARTERED_IN - the one-way relationship between companies and city (i.e. companies are headquartered in a city)
##### 2. LOCATED_IN - the one-way relationship between city and state (i.e. cities are located in a state)
##### 3. PART_OF - the one-way relationship between state and country (i.e. states are part of a country)


The resulting graph database has 27 nodes.

# Queries
## 1. Which companies are located in the state (not city) of Tokyo (via 2nd degree relation)?

In [47]:
query4 = ("MATCH (state:State {name: 'Tokyo'})-[*2]"
           "-(company:Company) RETURN DISTINCT company.name as companies"
          )
result4 = session.run(query4)   
print("The company located in the state of Tokyo is " + str([record["companies"] for record in result4]))

The company located in the state of Tokyo is ['Sony']


Here, I used the MATCH clause and matched the name property of the State label to 'Tokyo.' Ultimately, I returned the name property of the Company label based on the 2nd degree relationship between State and Company. The query result was stored in result4 and the output was 'Sony,' which is HEADQUARTERED_IN the City 'Tokyo', which in turn is LOCATED_IN the State 'Tokyo'. (i.e. 2nd degree relationship)

## 2. Which companies have been founded in the 1990s? (i.e., Between 1990 and 1999)

In [100]:
query5 = ("MATCH (company:Company)"
        "WHERE 1990 <= company.year <= 1999"
        "RETURN company.name as companies")
result5 = session.run(query5)   
print("Companies founded in the 1990s are " + str([record["companies"] for record in result5]))

Companies founded in the 1990s are ['Amazon', 'Google', 'Alibaba']


Here, I ran the query using the MATCH clause based on the Company node label and set a constraint for the 'year' property of the Company node label between 1990 and 1999 using the WHERE function.  Ultimately I returned the 'name' property of the Company node label based on the set constraint and stored the result in result5. The query ultimately resulted in a list containing the names of all the companies that were founded in the 90s.

## 3. What is the average number of employees for all companies in China and the USA in the graph? (via 3rd degree relation) (Use aggregate queries)

In [213]:
query6 = ("MATCH (country:Country {name:'USA'}) -[*3]"
           "-(company:Company) RETURN avg(company.employees) as employees"
          )
result6 = session.run(query6)   
[record["employees"] for record in result6]


[436694.0]

In this query I was unable to figure out how to include 'China' in my MATCH clause - but for partial credit I ran the aggregate 'avg' function to return the average number of employees at companies located in 'USA' based on the third degree relationship between the Country and Company.

## 4. Which companies are headquartered in the USA (via 3rd degree relation)?

In [106]:
query7 = ("MATCH (country:Country {name: 'USA'})-[*3]"
           "-(company:Company) RETURN DISTINCT company.name as companies"
          )
result7 = session.run(query6)   
print("Companies headquartered in the USA are " + str([record["companies"] for record in result6]))

Companies headquartered in the USA are ['Google', 'Apple', 'Amazon', 'Microsoft']


Here, I used the MATCH function to match the name property of the Country node label with 'USA' and returned the name property of the Company label based on the 3rd degree relationship between Country and Company. The query resulted in the list ['Google', 'Apple', 'Amazon', 'Microsoft'], i.e. all companies that are HEADQUARTERED_IN a certain corresponding City, which are located in their corresponding States, all of which are located in the country 'USA'. (i.e. 3rd degree relationship)

## 5. What is the total annual revenue of the companies in USA (via 3rd degree relation)?

In [211]:
query8 = ("MATCH (country:Country {name:'USA'}) -[*3]"
           "-(company:Company) RETURN sum(company.revenue) as revenue"
          )
result8 = session.run(query8)   
print("Total annual revenue for companies in USA is " + str([record["revenue"] for record in result8]) + " Million USD")

Total annual revenue for companies in USA is [1982.0] Million USD


Here I matched the name property of the Country node label to 'USA' in order to query the the USA node - and using the aggregate 'sum' function, I have returned the sum of all the revenues of companies headquartered in USA via third degree relation between Company and Country. The unaggregated query would return the list of all company revenues, whereas the aggregated query takes the sum of all the list objects. 